# Vehicle Registration Data Exploration

This notebook is for exploring the vehicle registration data schema and testing queries.


In [ ]:
import pandas as pd
import os
import sqlite3
from pathlib import Path

# Set up paths
data_dir = Path('../data')
db_path = Path('../db/vehicles.db')

# List all CSV files
csv_files = list(data_dir.glob('*.csv'))
print(f"Found {len(csv_files)} CSV files:")
for file in csv_files:
    print(f"  - {file.name}")

# Load and explore the first file to understand structure
print("\n" + "="*50)
print("EXPLORING FIRST FILE STRUCTURE")
print("="*50)

df_sample = pd.read_csv(csv_files[0], nrows=5)  # Load just first 5 rows
print(f"File: {csv_files[0].name}")
print(f"Shape: {df_sample.shape}")
print(f"Columns: {list(df_sample.columns)}")
print("\nFirst few rows:")
print(df_sample.head())
print(f"\nData types:")
print(df_sample.dtypes)


In [ ]:
# Compare column structures across all files
print("="*50)
print("COMPARING COLUMN STRUCTURES")
print("="*50)

column_analysis = {}
for file in csv_files:
    df_temp = pd.read_csv(file, nrows=1)  # Just headers
    column_analysis[file.name] = list(df_temp.columns)
    print(f"{file.name}: {len(df_temp.columns)} columns")

# Check if all files have the same columns
all_columns = set()
for columns in column_analysis.values():
    all_columns.update(columns)

print(f"\nTotal unique columns across all files: {len(all_columns)}")
print("All columns:", sorted(all_columns))

# Check for differences
print("\nColumn differences between files:")
for file, columns in column_analysis.items():
    missing = all_columns - set(columns)
    extra = set(columns) - all_columns
    if missing or extra:
        print(f"{file}:")
        if missing:
            print(f"  Missing: {missing}")
        if extra:
            print(f"  Extra: {extra}")
    else:
        print(f"{file}: ✓ All columns match")


In [ ]:
# Check data quality issues
print("="*50)
print("DATA QUALITY ANALYSIS")
print("="*50)

# Load a sample from each file to check for issues
for file in csv_files[:2]:  # Check first 2 files
    print(f"\nAnalyzing {file.name}:")
    df_temp = pd.read_csv(file, nrows=1000)  # Sample 1000 rows
    
    print(f"  Shape: {df_temp.shape}")
    print(f"  Missing values per column:")
    missing = df_temp.isnull().sum()
    for col, count in missing.items():
        if count > 0:
            print(f"    {col}: {count} ({count/len(df_temp)*100:.1f}%)")
    
    # Check for potential date columns
    date_cols = [col for col in df_temp.columns if any(word in col.lower() for word in ['date', 'time', 'year', 'month'])]
    if date_cols:
        print(f"  Potential date columns: {date_cols}")
        for col in date_cols:
            print(f"    {col} sample values: {df_temp[col].head(3).tolist()}")
    
    # Check for numeric columns
    numeric_cols = df_temp.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        print(f"  Numeric columns: {list(numeric_cols)}")
